In [3]:
import numpy as np
import matplotlib.pyplot as plt
import stim
import qutip
from stim import PauliString


from optimal_codes import *

# setup autoreload 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
class SeparablePauliNoise:
    def __init__(self, p_x, p_y, p_z, num_qubits=1):
        self.p_x = p_x
        self.p_y = p_y
        self.p_z = p_z
        self.num_qubits = num_qubits

    @property 
    def pvec_1q(self):
        px = self.p_x
        py = self.p_y
        pz = self.p_z
        return np.array([1-px-py-pz, px, py, pz])

    def pvec_joint(self):
        # tensor the pvec_1 for each qubit
        pvec = self.pvec_1q
        pvec_joint = np.array([1])
        for i in range(self.num_qubits):
            pvec_joint = np.kron(pvec_joint, pvec)
        return pvec_joint

    def sample_error(self):
        pvec = self.pvec_joint()
        return random_nqb_paulistring(pvec=pvec[1:], num_qubits=self.num_qubits)

In [39]:
np.concatenate?

Docstring:
concatenate((a1, a2, ...), axis=0, out=None, dtype=None, casting="same_kind")

Join a sequence of arrays along an existing axis.

Parameters
----------
a1, a2, ... : sequence of array_like
    The arrays must have the same shape, except in the dimension
    corresponding to `axis` (the first, by default).
axis : int, optional
    The axis along which the arrays will be joined.  If axis is None,
    arrays are flattened before use.  Default is 0.
out : ndarray, optional
    If provided, the destination to place the result. The shape must be
    correct, matching that of what concatenate would have returned if no
    out argument were specified.
dtype : str or dtype
    If provided, the destination array will have this dtype. Cannot be
    provided together with `out`.

    .. versionadded:: 1.20.0

casting : {'no', 'equiv', 'safe', 'same_kind', 'unsafe'}, optional
    Controls what kind of data casting may occur. Defaults to 'same_kind'.

    .. versionadded:: 1.20.0

Returns

In [40]:
dist = SeparablePauliNoise(0.1, 0.1, 0.1, num_qubits=5)

In [94]:
dist.sample_error()

'_X___'

In [ ]:
def make_bs_stabs(n, m):
    """
    Make the x-stabilizers for the Bacon-Shor code in the stim PauliString format.

    For n columns, there are n-1 X-stabilizers. For m rows, there are m-1 Z-stabilizers.

    The X stabilizers are the columns, as in 

    * X X * * 
    * X X * *
    * X X * *
    * X X * *
    * X X * *

    The Z stabilizers are the rows, as in

    Z Z Z Z Z Z
    Z Z Z Z Z Z
    * * * * * *
    * * * * * *
    * * * * * *

    Args:
        n (int): The number of rows in the Bacon-Shor code.
        m (int): The number of columns in the Bacon-Shor code.
    """
    assert n % 2 == 1
    assert m % 2 == 1
    num_x_stabs = (m-1)
    num_z_stabs = (n-1)

    x_stabs = []
    z_stabs = []

    # make the X-stabilizers
    # an X-stabilizer is a column of Xs, stored as a PauliString

    for k in range(num_x_stabs):
        x_stab_binary_mat = np.zeros((n, m))
        x_stab_binary_mat[:, k] = 1
        x_stab_binary_mat[:, k+1] = 1
        # now change the binary matrix in an n*m character string
        x_stab_str = ""
        for i in range(n):
            for j in range(m):
                if x_stab_binary_mat[i, j] == 1:
                    x_stab_str += "X"
                else:
                    x_stab_str += "_"
        x_stabs.append(PauliString(x_stab_str))

    # make the Z-stabilizers
    # a Z-stabilizer is a row of Zs, stored as a PauliString

    for k in range(num_z_stabs):
        z_stab_binary_mat = np.zeros((n, m))
        z_stab_binary_mat[k, :] = 1
        z_stab_binary_mat[k+1, :] = 1
        # now change the binary matrix in an n*m character string
        z_stab_str = ""
        for i in range(n):
            for j in range(m):
                if z_stab_binary_mat[i, j] == 1:
                    z_stab_str += "Z"
                else:
                    z_stab_str += "_"
        z_stabs.append(PauliString(z_stab_str))
    

    return x_stabs, z_stabs


    



In [107]:
make_bs_stabs(5, 11)

([stim.PauliString("+XX_________XX_________XX_________XX_________XX_________"),
  stim.PauliString("+_XX_________XX_________XX_________XX_________XX________"),
  stim.PauliString("+__XX_________XX_________XX_________XX_________XX_______"),
  stim.PauliString("+___XX_________XX_________XX_________XX_________XX______"),
  stim.PauliString("+____XX_________XX_________XX_________XX_________XX_____"),
  stim.PauliString("+_____XX_________XX_________XX_________XX_________XX____"),
  stim.PauliString("+______XX_________XX_________XX_________XX_________XX___"),
  stim.PauliString("+_______XX_________XX_________XX_________XX_________XX__"),
  stim.PauliString("+________XX_________XX_________XX_________XX_________XX_"),
  stim.PauliString("+_________XX_________XX_________XX_________XX_________XX")],
 [stim.PauliString("+ZZZZZZZZZZZZZZZZZZZZZZ_________________________________"),
  stim.PauliString("+___________ZZZZZZZZZZZZZZZZZZZZZZ______________________"),
  stim.PauliString("+__________________